In [1]:
import networkx as nx
from collections import defaultdict
import random

In [11]:
# Values of hyper parameters as mentioned in the research paper

alpha = 0.7    # Used in similarity calculation
beta = 0.3    # Used in similarity calculation

I = 50    # Maximum number of cycles/iterations
K = 1    # Number of selected nodes
A = K    # Number of ants

tau0 = 1    # The initial amount of pheromone
rho = 0.05    # Pheromone evaporation coefﬁcient
eta = 1    # Relative importance of the pheromone value
psi = 1    # Relative importance of the proﬁt value
gamma = 1    # Relative importance of the similarity value
theta = 1    # Threshold

cost = 0.1    # Cost of selecting a node
r = 1    # profit by selecting a node/customer

In [12]:
# Helper functions

# Returns first order neighbours of node in a graph
def first_order_neighbours(graph: nx.graph, node):
    return set(graph.adj[node])

# Returns second order neighbours of node in a graph
def second_order_neighbours(graph: nx.graph, node):
    ret = set()
    
    for u in graph[node]:
        for v in graph[u]:
            if v != node:
                ret.add(v)
    
    return ret

# Initializes the pheromone amount
def initialize_pheromones(graph: nx.graph, initial_pheromone_value: float):
    for u, v in graph.edges:
        graph[u][v]['pheromone'] = initial_pheromone_value

# Updates the pheromone amount of all edges
def update_pheromones(graph: nx.graph, fitness_of_ants: list, evaporation_coefficient: float):
    fitness_along_edge = {}

    for fitness, ant in fitness_of_ants:
        for u in ant:
            for v in ant:
                fitness_along_edge[(u, v)] = fitness_along_edge.get((u, v), 0) + fitness
    
    for u, v in graph.edges:
        graph[u][v]['pheromone'] = (1 - evaporation_coefficient) * graph[u][v]['pheromone'] + fitness_along_edge.get((u, v), 0)

In [13]:
# Methods related to similarity between nodes      

# Finds similarity between all pairs of nodes in the graph
def find_similarities_between_all_pairs(graph: nx.graph):
    for u in graph.nodes():
        for v in graph.nodes():
            similarity_u_v = 1

            if u != v and u not in graph[v]:
                common_first_order_neighbours = first_order_neighbours(graph, u).union(first_order_neighbours(graph, v))
                all_first_order_neighbours = first_order_neighbours(graph, u).intersection(first_order_neighbours(graph, v))

                common_second_order_neighbours = second_order_neighbours(graph, u).union(second_order_neighbours(graph, v))
                all_second_order_neighbours = second_order_neighbours(graph, u).intersection(second_order_neighbours(graph, v))

                s_first = len(common_first_order_neighbours) / len(all_first_order_neighbours)
                s_second = len(common_second_order_neighbours) / len(all_second_order_neighbours)

                similarity_u_v = alpha * s_first + beta * s_second
            
            # graph[u][v]['similarity'] = similarity_u_v
            graph.add_edge(u, v, similarity = similarity_u_v)

# Returns the similarity value of a given set of nodes
def similarity_of_set_of_nodes(graph: nx.graph, node_set: set):
    s = 0
    for u in node_set:
        for v in node_set:
            s += graph[u][v]['similarity']
    
    return s

In [14]:
# Linear threshold model

def influence_of_activated_nodes(graph: nx.graph, seeds: set, threshold: float):
    influnces = list(seeds)
    queue = influnces[:]
    pre_node_record = defaultdict(float)

    while len(queue) != 0:
        node = queue.pop(0)
        
        for element in graph[node]:
            if element not in influnces:
                pre_node_record[element] = pre_node_record[element] + graph[node][element].get('weight', 0)
                
                if pre_node_record[element] >= threshold:
                    influnces.append(element)
                    queue.append(element)
                    
    return influnces

In [15]:
# Positions the ant -> finds a path on the graph of length 'set_size'

def generate_ant(graph: nx.graph, set_size: int, threshold: float):
    n = len(graph.nodes())
    
    all_nodes = set(range(1, n+1))
    selected = set()

    node = random.randint(1, n)
    selected.add(node)

    for _ in range(set_size - 1):
        nodes_similarity = []

        for u in all_nodes - selected:
            pr = influence_of_activated_nodes(graph, selected + set(u), threshold)
            s = similarity_of_set_of_nodes(graph, selected + set(u))
            tau = graph[node][u]['pheromone']

            sim = pow(tau, eta) * pow(pr, psi) / pow(s, gamma)
            nodes_similarity.append((sim, u))
        
        node = max(nodes_similarity)[1]
        selected.add(node)
    
    return selected

In [16]:
# Returns the fitness value of the given ant
def get_fitness_of_ant(graph: nx.graph, ant: set, threshold: float):
    no_of_nodes_influenced = len(influence_of_activated_nodes(graph, ant, threshold))
    K = len(ant)

    pr = r * no_of_nodes_influenced - cost * K
    s = similarity_of_set_of_nodes(graph, ant)

    fitness = pow(pr, psi) / pow(s, gamma)
    return fitness

In [17]:
def IMOACO(graph: nx.graph, K: int, threshold: float, evaporation_coefficient: float):
    # Finding similarities between every pair of nodes in graph
    find_similarities_between_all_pairs(graph)

    # Initializing the pheromone amount
    initialize_pheromones(graph, tau0)
    
    for t in range(I):
        fitness_of_ants = []

        for i in range(A):
            ant = generate_ant(graph, K, threshold)

            fitness = get_fitness_of_ant(graph, ant, threshold)
            fitness_of_ants.append((fitness, ant))
        
        update_pheromones(graph, fitness_of_ants, evaporation_coefficient)
            
        fitness_of_ants.sort()
        best = fitness_of_ants[-1][1]

    return best

In [2]:
# graph is the social network
graph = nx.Graph()

# Constructing the graph
graph.add_node(1)
graph.add_node(2)
graph.add_node(3)

# Adding random weights to edges, should be replaced by reading the input
for u in graph.nodes():
    for v in graph.nodes():
        graph.add_edge(u, v, weight = random.random())

IMOACO(graph, K, theta, rho)